In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import gc
os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Experian_Email_Openers_Group'

In [2]:
output_folder="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Experian_Email_Openers_Group/output/"
try:
    os.stat(output_folder)
except:
    os.mkdir(output_folder)

In [3]:
Experian_data_HH=pd.read_table("/home/jian/BigLots/Experian/MS_Experian_HH_Full.zip",
                            compression="zip",dtype=str,sep="|",
                            usecols=['Customer_Num','Email_Address_hash','multi_cluster','multi_cluster_desc'])

Experian_data_GEO=pd.read_table("/home/jian/BigLots/Experian/MS_Experian_GEO_Full.zip",
                            compression="zip",dtype=str,sep="|",
                            usecols=['Customer_Num','Email_Address_hash','multi_cluster','multi_cluster_desc'])
Experian_data=Experian_data_HH.append(Experian_data_GEO)

del Experian_data_HH
del Experian_data_GEO

gc.collect()

7

In [4]:
summary_cluster=Experian_data.groupby(['multi_cluster_desc','multi_cluster'])['Email_Address_hash'].count().to_frame().reset_index()
summary_cluster=summary_cluster.rename(columns={"Email_Address_hash":"ID_counts"})
summary_cluster

,multi_cluster_desc,multi_cluster,ID_counts
0,Holiday Shoppers,6,926878
1,Home Focused,5,2230946
2,Infrequent Families,3,3225551
3,Rising Hispanics,2,2258858
4,Spending Fanatics,8,974928
5,Sporadic Seniors,7,4202032
6,Starters in Need,1,2329314
7,Urban Affluence,4,1809867


In [5]:
selected_clusters=set(['Home Focused','Spending Fanatics','Holiday Shoppers','Urban Affluence','Rising Hispanics'])
Experian_data=Experian_data[Experian_data['multi_cluster_desc'].isin(selected_clusters)]
Experian_data.shape

(8201477, 4)

In [6]:
Email_Opener=pd.read_table("/home/jian/BigLots/Email_Openers_received_20190405/MediaStorm_90_days_Opens_Customer_extract.zip",
                          dtype=str,sep=",",usecols=['email_address_hashed'])
Email_Opener['Email_Type']="Email Openers"
Email_Opener=Email_Opener.rename(columns={"email_address_hashed":"Email_Address_hash"})
Email_Opener=Email_Opener.drop_duplicates()

In [7]:
Experian_data=pd.merge(Experian_data,Email_Opener,on="Email_Address_hash",how="left")
Experian_data['Email_Type']=Experian_data['Email_Type'].fillna("Non-Opener")
Experian_data.shape

(8201477, 5)

In [8]:
Experian_data.head(20)

,Customer_Num,Email_Address_hash,multi_cluster,multi_cluster_desc,Email_Type
0,35340040,bf833c5d4d0ce58328e1fc4cd4ada7ff73e945b12c4d65...,8,Spending Fanatics,Non-Opener
1,13299531,01f6f108afdca3d1a45fe60d6df2ec1ea58fb764043dec...,6,Holiday Shoppers,Email Openers
2,21511658,c512a59c221fc079dd5764dd793f2a55f3d39f000a120d...,8,Spending Fanatics,Non-Opener
3,25264018,067e6f345391802f3506353509961d731261360ffe4036...,4,Urban Affluence,Non-Opener
4,16617430,f5613f432983bb27834fc46c03e5c7fae8dba61ed3c747...,6,Holiday Shoppers,Email Openers
5,48289389,ac1f6d49915c7e114319f097e830494e670f7d01cfa797...,5,Home Focused,Non-Opener
6,25792149,00c19a3944806761f6254de1a7c8a2677a9c4a5d8d739d...,4,Urban Affluence,Email Openers
7,18202101,0e4d8a5a60962ba182f2c74bc00a61c0b531e6cdf60426...,4,Urban Affluence,Non-Opener
8,13016641,170677876eec7776a72145de7c8cbb0ae2938672a7e62d...,6,Holiday Shoppers,Non-Opener
9,37014094,0f8a528337a8c37f78eaf4d7a5165dfa75783599df9330...,5,Home Focused,Email Openers


In [18]:
Experian_output=pd.DataFrame()
import random
random.seed(20190410)
for ind,group in Experian_data.groupby(['multi_cluster_desc','Email_Type']):
    total_rows=len(group)
    group=group.reset_index()
    del group['index']
    group=group.reset_index()
    
    control_group_list=random.sample(range(total_rows),int(np.floor(total_rows*0.05)))
    control_group_df=pd.DataFrame({"index":control_group_list})
    control_group_df['T_C_group']="Control"
    group=pd.merge(group,control_group_df,on="index",how="left")
    group['T_C_group']=group['T_C_group'].fillna("Test")
    del group['index']
    Experian_output=Experian_output.append(group)
    
print(Experian_output.shape)

Experian_output.head(2)

(8201477, 6)


,Customer_Num,Email_Address_hash,multi_cluster,multi_cluster_desc,Email_Type,T_C_group
0,13299531,01f6f108afdca3d1a45fe60d6df2ec1ea58fb764043dec...,6,Holiday Shoppers,Email Openers,Test
1,16617430,f5613f432983bb27834fc46c03e5c7fae8dba61ed3c747...,6,Holiday Shoppers,Email Openers,Test


In [29]:
Experian_output['segment']=Experian_output['T_C_group']+" | "+Experian_output['multi_cluster_desc']+" | "+Experian_output['Email_Type']


In [28]:
Experian_output.tail(2)

,Customer_Num,Email_Address_hash,multi_cluster,multi_cluster_desc,Email_Type,T_C_group,segment,customer_id_hashed
1192185,43223881,504911b893b063b1db81b8fc86941c960a7fef95e69178...,4,Urban Affluence,Non-Opener,Test,Test|Urban Affluence|Non-Opener,5e1c4eb46043580ac3a48eaff9cd313be21ee0f6f25310...
1192186,18111854,e10b077d8b09594b69d2da6aec7c9a9475f49afcc352ea...,4,Urban Affluence,Non-Opener,Test,Test|Urban Affluence|Non-Opener,8c292deac20622b0ea7b57d91ba5da4c77c1db066acfc1...


In [27]:
import hashlib
Experian_output['customer_id_hashed']=Experian_output['Customer_Num'].apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest())


In [36]:
for seg,group in Experian_output.groupby('segment'):
    group=group[['customer_id_hashed','Email_Address_hash','segment']]
    file_name=seg[0]+"_"+seg.split("|")[1]+"_"+seg.split("|")[2]
    file_name=file_name.replace(" ","")
    group.to_csv(output_folder+"BL_Experian_segment_of_"+file_name+"_"+str(datetime.datetime.now().date())+".csv",index=False)

In [37]:
Experian_output_summary=Experian_output.groupby(['segment','multi_cluster_desc','Email_Type','T_C_group'])['Email_Address_hash'].count().to_frame().reset_index()
Experian_output_summary=Experian_output_summary.rename(columns={"Email_Address_hash":"Email_Counts"})


In [38]:
Experian_output_summary.to_csv(output_folder+"BL_Experian_segment_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
